In [2]:
!pip install metric_learn

from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from metric_learn import MLKR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from prettytable import PrettyTable
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF


drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/EE 559/parkinsons_updrs.data')
print(data)

y=data[['motor_UPDRS', 'total_UPDRS']]
data = data.drop('motor_UPDRS', axis=1)
X = data.drop('total_UPDRS', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                   random_state=104,
                                   test_size=0.30,
                                   shuffle=True)

# Standardize Features
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()


y_train1 = y_train.iloc[:,0]
y_train2 = y_train.iloc[:,1]

y_test1 = y_test.iloc[:,0]
y_test2 = y_test.iloc[:,1]

kernel = 1.0 * RBF(1.0)

# 5 fold cross validation n_components  [5 10 15: p]
hyperparameter_score_list = []
# Cross Validation
for M in [5, 10, 15, 22]:
    if M != 22:
      mlkr = MLKR(n_components = M, init = 'pca')
    else:
      mlkr = MLKR()
    mklr_predictor = mlkr.fit(X_train,y_train1)
    x_train_transform = mklr_predictor.transform(X_train)
    x_test_transform = mklr_predictor.transform(X_test)

    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(x_train_transform, y_train1)
    y_pred = gpr.predict(x_test_transform)

    r_score = r2_score(y_test1, y_pred)
    scores = cross_val_score(gpr, X_train, y_train1, cv=5)
    mean_score = np.mean(scores)

    hyperparameter_score_list.append([M, mean_score, gpr.score, r_score])

myTable = PrettyTable(["p", "Avg accuracy", "Train R2", "Test R2"])
for row in hyperparameter_score_list:
    myTable.add_row([row[0], round(row[1],3), row[2], row[3]])
print(myTable)

# 5 fold cross validation n_components  [5 10 15: p]
hyperparameter_score_list = []
# Cross Validation
for M in [5, 10, 15, 22]:
    if M != 22:
      mlkr = MLKR(n_components = M, init = 'pca')
    else:
      mlkr = MLKR()
    mklr_predictor = mlkr.fit(X_train,y_train2)
    x_train_transform = mklr_predictor.transform(X_train)
    x_test_transform = mklr_predictor.transform(X_test)

    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(x_train_transform, y_train2)
    y_pred = gpr.predict(x_test_transform)

    r_score = r2_score(y_test2, y_pred)
    scores = cross_val_score(gpr, X_train, y_train2, cv=5)
    mean_score = np.mean(scores)

    hyperparameter_score_list.append([M, mean_score, gpr.score, r_score])

myTable = PrettyTable(["p", "Avg accuracy", "Train R2", "Test R2"])
for row in hyperparameter_score_list:
    myTable.add_row([row[0], round(row[1],3), row[2], row[3]])
print(myTable)


'''
sklearn’s neural network implementation
single layer
two output

including
the number of neurons in the hidden and output layers
method of optimization
type of activation functions
and the L2 “regularization” parameter etc.
Remember to standardize yourfeatures. Report your R2 on both training and test sets. (20 pts)
'''
regr = MLPRegressor(hidden_layer_sizes=(5,), early_stopping=False, random_state=1, max_iter=500).fit(X_train, y_train)
regr.predict(X_test[:2])
regr.score(X_test, y_test)



'''
early-stopping=True. Research what early stopping is,
and compare the performance of your network on the test set with the previous
network. You can leave the validation-fraction as the default (0.1) or change it
to see whether you can obtain a better model.
R2 on both training and test sets
'''


regr = MLPRegressor(hidden_layer_sizes=(5,), early_stopping=True, random_state=1, max_iter=500).fit(X_train, y_train)
regr.predict(X_test[:2])
regr.score(X_test, y_test)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      subject#  age  sex  test_time  motor_UPDRS  total_UPDRS  Jitter(%)  \
0            1   72    0     5.6431       28.199       34.398    0.00662   
1            1   72    0    12.6660       28.447       34.894    0.00300   
2            1   72    0    19.6810       28.695       35.389    0.00481   
3            1   72    0    25.6470       28.905       35.810    0.00528   
4            1   72    0    33.6420       29.187       36.375    0.00335   
...        ...  ...  ...        ...          ...          ...        ...   
5870        42   61    0   142.7900       22.485       33.485    0.00406   
5871        42   61    0   149.8400       21.988       32.988    0.00297   
5872        42   61    0   156.8200       21.495       32.495    0.00349   
5873        42   61    0   163.7300       21.007       32.007    0.00281   
5874        42   61    0   170.7300

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


0.4668189257814127